# **1. Question or problem definition.**

โดยปกติ หนังที่มีคนดูเยอะ ไม่ได้หมายความว่าเป็นหนังที่ดีเสมอไป คนอาจจะสนใจเยอะเพราะการแอ็คชั่น หรือ กราฟฟิกที่ดี
และหนังที่ดี ก็ไม่ได้หมายถึงว่ามีคนดูเยอะเสมอไป เพราะเนื้อเรื่องอาจจะเด่น น่าสนใจ แต่ไม่ถูกฉายในโรงภาพยนต์ในช่วงฮอตไทม์


ดังนั้น เราจึงทำการหาท็อปหนัง10เรื่องที่ มีทั้งความนิยมที่สูง(popularity) และ คะแนนโหวตเฉลี่ยจากคนที่ชมและให้คะแนนตามความชอบว่าเป็นหนังที่ดีหรือไม่(rating) มาคำนวณเพื่อหาหนังที่ดีที่สุด10เรื่อง โดยใช้ข้อมูลจากทั้งสองตัวแปร
มาเป็นตัวแปรในสูตรเพื่อหา Weighted_rating** เพื่อให้ได้ภาพยนตร์ที่มีความสมดุลของคะแนนและความนิยม

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  **2.Acquire training and testing data**
จากหัวข้อที่เราได้เลือกมานั้นทำให้เราต้องการข้อมูลเพื่อนำมาเปรียบเทียบ โดยจะมีการนำข้อมูลเข้ามาใช้ ดังนี้

    * id = รหัสของหนังที่เราจะเอามาเปรียบเทียบกัน
    * original_title = ชื่อหนังที่เราจะนำมาใช้
    * revenue = รายได้ของหนังที่ได้รับ
    * runtime = เวลาของหนังที่ฉายทั้งเรื่อง
    * rating = เรทติ้งของหนัง
    * vote_count = จำนวนโหวตทั้งหมดของผู้ชม
    * popularity = ความนิยมของหนังเมื่อเทียบกับจำนวนผู้ชม
    * Weighted_rating = เป็นค่าที่เข้าสูตรและคำนวณออกมาเพื่อนำไปเปรียบเทียบ


In [ ]:
#ทำการนำข้อมูลเข้าไปที่ตัวแปล movies
movie = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

#ทำการแสดงออกมาว่าข้อมูลเข้ามาอยู่ที่ตัวแปลแล้ว และที่แสดงส่วนท้ายเพราะจะได้รู้ว่ามีจำนวนข้อมูลอยู่กี่ตัว
movie.tail()

# **3. Wrangle, prepare, cleanse the data.**

จะเป็นขั้นตอนการเตรียมข้อมูลเพื่อที่จะทำให้ข้อมูลเตรียมพร้อมที่จะนำวิเคราะห์โดยจะเป็นการบอกว่าจะใช้ข้อมูลไหน ทำการเปลี่ยนชื่อ ทำตารางข้อมูลใหม่ ลบข้อมูลส่วนที่ไม่ใช้ ตรวจสอบข้อมูลว่ามีความผิดพลาดไหม
    

In [ ]:
#จะเห็นได้ว่ามีหัวของหมวดหมู่มากมายที่เราได้นำข้อมูลเข้ามาและจะมีส่วนที่เราไม่ได่ใช้ 
movie.columns

In [ ]:
##ทำการเช็ตดูว่าตัว column ที่เราใช้มึค่า null หรือไม่ ซึ่งถ้าค่าเป็น False จะไม่มีค่า null 
movie.isnull().any()

In [ ]:
#ทำการซ่อมข้อมูลที่ขาดหายไป ถ้าข้อมูลไหนมีค่า null จะเติมค่า 0 เข้าไปแทนที่
movie = movie.fillna(value=0, axis=1)

In [ ]:
#ทำการเช็คอีกครั้งหลังจาก ซึ่งถ้าไม่มี null จะเป็น False
movie.isnull().any()

In [ ]:
#เปลี่ยนชื่อจาก vote_average เป็น rating เพื่อให้อ่านแล้วเข้าใจง่าย และอ่านแล้วไม่สับสนกับ vote_count
movie = movie.rename(columns = {'vote_average' : 'rating'})
movie.head()

In [ ]:
#จะเห็นได้ถึงความแตกต่างของการโหวตของหนังแต่ละเรื่องว่าบางเรื่องมีแค่โหวคเดียว บางโหวตมีเป็นหมื่น
movie.groupby('vote_count')['original_title'].max()

# **4. Analyze, identify patterns, and explore the data.**



ทำการวิเคราะห์ว่าเราควรใช้ตัวไหนในการเปรียบเทียบ และเพื่อจะหาคำตอบของเป้าหมายของเรา
ซึ่งข้อมูลที่เราได้ทำการเลือกมาในการคำนวณมีดังนี้
* rating, 
* vote_count, 
* popularity 
และจะนำสิ่งที่เรามีไปคำนวนหาด้วยการใช้สูตรการคำนวนหา 
* Weighted_rating ซึ่งคือคะแนนที่คำนวณมาจากค่าทั้งหมดเพื่อหาหนังที่ดีที่สุด



และด้วยรูปแบบที่คะแนนออกมาเป็นTOP10 เราจึงได้ทำเป็นกราฟออกมาในตอนท้าย เพื่อให้ง่ายต่อการเข้าใจ

In [ ]:
#เราจึงจะทำการเลือกเฉพาะตัว columns ที่เราจะใช้
movies_df = movie[[ 'id','original_title','rating','vote_count','popularity']]
movies_df.head()

ทำการเพิ่มตัวแปลของชุดข้อมูลตัวใหม่เพื่อเลือกมาเฉพาะตัวที่เราจะเอามาคิดคำนวนเพื่อง่ายต่อการดู
>     1.original_title
>     2.rating
>     3.vote_count
>     4.popularity

In [ ]:
movies_df.info()

In [ ]:
movies_df.isnull().any()

In [ ]:
#ทำการกำหนดตัวแปลเพื่อทำการคำนวนค่า Weighted_rating
R = movies_df['rating']
v = movies_df['vote_count']
m = movies_df['vote_count'].quantile(0.7) #กำหนด 30% ของหนังที่ได้รับความนิยม
C = movies_df['rating'].mean()

In [ ]:
#เพื่ม columns Weighted_rating โดยการใช้สูตรคำนวน
movies_df['weighted_rating'] = ((R*v)+(C*m)) / (v+m)

In [ ]:
#ตารางหลังจากเพิ่ม column มาแล่ว
movies_df

**เป็นส่วนของการแสดงอันดับภาพยนตร์ ที่ใช้การเรียงลำดับด้วยค่า popularity**

In [ ]:
#ทำการดึงข้อมูลจากใน column มาใส่ใน weight_average_popularity
weight_average_popularity=movies_df.sort_values('popularity',ascending=False)
#กำหนด ขนาดของกราฟที่จะแสดง 
plt.figure(figsize=(12,6))
#เป็นการกำหนดข้อมูลใน column และขนาดการแสดงค่าของแกน x และ แกน y โดยใช้ข้อมูลจากตัวแปล weight_average_popularity ที่ดึงค่ามาจาก movies_df
axis1=sns.barplot(x=weight_average_popularity['popularity'].head(10), y=weight_average_popularity['original_title'].head(10), data=weight_average_popularity);
#กำหนดชื่อที่หัวของแผนภูมิ
plt.title('Best Movies by Popularity score', weight='bold')
#กำหนดชื่อในแนวแกน x
plt.xlabel('Popularity Score', weight='bold')
#กำหนดชื่อในแนวแกน y
plt.ylabel('Movie Title', weight='bold');

**แสดงอันดับของภาพยนตร์ที่ใช้ค่าของคะแนน weighted_rating**

In [ ]:
#ทำการดึงข้อมูลจากใน column มาใส่ใน weighted_average
weighted_average=movies_df.sort_values('weighted_rating',ascending=False)
#กำหนด ขนาดของกราฟที่จะแสดง 
plt.figure(figsize=(12,6))
#เป็นการกำหนดข้อมูลใน column และขนาดการแสดงค่าของแกน x และ แกน y โดยใช้ข้อมูลจากตัวแปล weighted_average ที่ดึงค่ามาจาก movies_df
axis1=sns.barplot(x=weighted_average['weighted_rating'].head(10), y=weighted_average['original_title'].head(10), data=weighted_average)
plt.xlim(4, 10)
#กำหนดชื่อที่หัวของแผนภูมิ
plt.title('Best Movies by weighted_rating', weight='bold')
#กำหนดชื่อในแนวแกน x
plt.xlabel('Weighted Average Score', weight='bold')
#กำหนดชื่อในแนวแกน y
plt.ylabel('Movie Title', weight='bold');

**คะแนนส่วนใหญ่จะกระจุกอยู่ที่ 5-7 คะแนนจึงจะทำการแสดงคะแนนมากสุด น้อยสุด และคะแนนเฉลี่ย******

In [ ]:
#กำหนด ขนาดของกราฟที่จะแสดง 
plt.subplots(figsize=(12,10))
#เป็นการบอกว่าจะใช่แผนภูมิแบบไหนและดึงข้อมูลมาจากไหนและตัวไหน กำหนดสี และตวามเข้มของสี
n, bins, patches = plt.hist(movies_df['rating'], 30, density=1, color='orange', alpha=0.8)
#กำหนดข้อมูลที่จะมาแสดงในแกน x
plt.xlabel('rating')
#กำหนดข้อมูลที่จะมาแสดงในแกน y
plt.ylabel('Occurence')
#กำหนดชื่อที่หัวของแผนภูมิ
plt.title('Distribution of voter average')
plt.grid(True)
plt.show()

In [ ]:
data1=movies_df.loc[:,['rating','vote_count','popularity']]
data1.plot()
plt.show()

กราฟแสดงความหนาแน่นของคะแนนว่าคะแนนส่วนใหญ่อยู่ตรงไหน เมื่อนำrating และ popularity ของผู้ชมส่วนใหญ่มาเทียบกัน

In [ ]:
data1.plot(kind='scatter',x='rating',y='popularity',alpha=0.8)
plt.show()

คะแนนสูงสุด ต่ำสุด และ คะแนนเฉลี่ยที่เรเห็นอยู่ในกราฟที่กระจุกกันว่ามีค่าเท่าไหร่

In [ ]:
print("Minimum of Ratings:", round(min(movies_df['rating']),2))
print("Maximum of Ratings:", round(max(movies_df['rating']),2))
print("Average of Ratings:", round(np.mean(movies_df['rating']),2))
print("Variance of Ratings:",round(np.var(movies_df['rating']),2))

# 5. Model, predict and solve the problem.

ปัญหาของเราคือ ในData set แต่ละตัวนั้นจะมีแยกกันเป็นสองส่วน ส่วนแรกคือข้อมูลที่เป็นความนิยม หรือ popularity
ส่วนที่สองนั้นคือ ข้อมูลที่เป็นความชอบ หรือก็คือ rating 
ซึ่งโดยปกตินั้น หนังที่ดี ไม่ได้หมายความว่าคนจะดูเยอะเสมอไป หนังที่ดี อาจจะมีเนื้อเรื่องที่ดี แต่คนไม่ค่อยรู้จักเพราะมันเก่าแล้ว
แต่หนังที่นิยมและคนรู้จักนั้น อาจไม่ได้มีเนื้อเรื่องที่ดี หรือ เนื้อหาที่ดี แต่อาจจะมีภาพที่สวย คนอาจจะติดภาพของนักแสดง การแสดง เอฟเฟค ความตลก หรือ แอ็คชั่นที่สวยงามและดึงดูดคนดูก็ได้

ดังนั้น เนื่องจากเราต้องการหาค่าที่มีทั้งpopularity และ weighted rating เราจึงได้ทำการMergeข้อมูลมาเพื่อใช้ในการทำVisualize data
และเขียนโค้ดที่นำค่าทั้งสองมาใช้ร่วมกัน เพื่อหาหนังที่มีทั้งความนิยมและคะแนนโหวตจากผู้ชมที่สูง ทำให้ได้Visualize data แบบใหม่ขึ้นมา ซึ่งเป็นการเจาะจงตามเป้าหมายที่เราเล็งเอาไว้

ใช้ model การทำ regression คือจะใช้ linear regression ในการดูค่าความเปลี่ยนแปลงระหว่าง popularity กับ vote_count ที่เราได้จากการคำนวนมา

In [ ]:
features = ['popularity', 'vote_count']
target = ['rating']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 

# Importing modules
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
import json

เราจะทำการแบ่งข้อมูลเพื่่อทำการ train และ test

In [ ]:
#เราจะทำการsplitting data set into training and test data set in 0.7/0.3
train, test = train_test_split(movies_df,test_size=0.30)
train.head()

In [ ]:
#นำข้อมูลที่ต้องการใส่ลงใน train และ test
X_train = train[features].dropna()
y_train = train[target].dropna()
X_test = test[features].dropna()
y_test = test[target].dropna()

In [ ]:
#ทำการแสดงขนาดส่วนของ train และ test
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:
from sklearn import linear_model# compute classification accuracy for the linear regression model
from sklearn import metrics # for the check the error and accuracy of the model
lin = linear_model.LinearRegression()
# train the model on the training set
lin.fit(X_train, y_train)

In [ ]:
lin_score_train = lin.score(X_test, y_test)
lin_score_test = lin.score(X_train, y_train)

ผลลัพธ์ของการ Training และ Testing

In [ ]:
print("Training score: ",lin_score_train)
print("Testing score: ",lin_score_test)

In [ ]:
y_hat = lin.predict(X_test)
y_hat

# ** 6.Visualize, report, and present the problem solving steps and final solution.**
หลังจากที่เราได้ทำการกรองข้อมูลที่เราต้องการมาแล้วทั้งสองตัว เราก็มาเข้าสูตรเพื่อคำนวณหาหนังที่ดีที่สุดทั้ง10เรื่อง ที่มีทั้งความนิยม(popularity) และ คะแนนที่ดี (weighted rating)
ซึ่งผลลัพท์จะได้ออกมาเป็นตัวแปร Hybrid score ซึ่งจะทำให้เราได้ทราบเป้าหมายของเราซึ่งก็คือหนังที่ดีที่สุดจากทั้งด้านผู้ชม และ คะแนนนิยม นั้นมีเรื่องอะไรบ้างซึ่งสามารถนำไปใช้ประกอบการตัดสินใจเพื่อลงทุนในการทำหนัง หรือ เป็นแนวทางในการตัดสินใจได้ในอนาคต




In [ ]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
scaled_scores = pd.DataFrame(scalar.fit_transform(movies_df[['popularity','weighted_rating']]), 
                             columns=['popularity_scaled','weighted_rating_scaled'])
movies_scaled = movies_df.copy()
movies_scaled[['popularity_scaled','weighted_rating_scaled']] = scaled_scores
movies_scaled

In [ ]:
#ทำการดึงข้อมูลจากใน column มาใส่ใน rating_average
rating_average=movies_scaled.sort_values('rating',ascending=False)
#กำหนด ขนาดของกราฟที่จะแสดง 
plt.figure(figsize=(12,6))
#เป็นการกำหนดข้อมูลใน column และขนาดการแสดงค่าของแกน x และ แกน y โดยใช้ข้อมูลจากตัวแปล weight_average ที่ดึงค่ามาจาก movies_df
axis1=sns.barplot(x=rating_average['rating'].head(10), y=rating_average['original_title'].head(10), data=rating_average)
plt.xlim(4, 10)
#กำหนดชื่อที่หัวของแผนภูมิ
plt.title('Best Movies by rating', weight='bold')
#กำหนดชื่อในแนวแกน x
plt.xlabel('Rating Average Score', weight='bold')
#กำหนดชื่อในแนวแกน y
plt.ylabel('Movie Title', weight='bold');

ทำการคำนวน เพื่อจะได้คะแนน scaled_scores แล้วจึงนำมาคำนวนเป็นครั้งสุดท้ายก่อนจะไปแสดงผล

In [ ]:
movies_scaled['score'] = (0.5 * movies_scaled['weighted_rating_scaled']) + \
                            (0.5 * movies_scaled['popularity_scaled'])

In [ ]:
#ทำการดึงข้อมูลจากใน column มาใส่ใน weight_average_score_hybrid
weight_average_score_hybrid=movies_scaled.sort_values('score',ascending=False)
#กำหนด ขนาดของกราฟที่จะแสดง 
plt.figure(figsize=(12,6))
#เป็นการกำหนดข้อมูลใน column และขนาดการแสดงค่าของแกน x และ แกน y โดยใช้ข้อมูลจากตัวแปล weight_average_score_hybrid ที่ดึงค่ามาจาก movies_df
axis1=sns.barplot(x=weight_average_score_hybrid['score'].head(10), y=weight_average_score_hybrid['original_title'].head(10), data=weight_average_score_hybrid);
#กำหนดชื่อที่หัวของแผนภูมิ
plt.title('Best Movies by Hybrid score', weight='bold')
#กำหนดชื่อในแนวแกน x
plt.xlabel('Hybrid Score', weight='bold')
#กำหนดชื่อในแนวแกน y
plt.ylabel('Movie Title', weight='bold');

# 7. Supply or submit the results.

จากผลลัพท์ข้างต้นจะทำให้เราได้ข้อมูลใหม่ซึ่งทำการจัดเรียงข้อมูลขึ้นมาใหม่จากการนำตัวแปรไปคำนวณ โดยจะได้ออกมาเป็นHybrid Score

HYBRID SCORE คือตัวแปรที่เราได้จากการทำโค้ดทั้งหมด ซึ่งก็คือหนังที่มีทั้งสองตัวแปรอยู่ร่วมกัน นั้นคือpopularity และ rating ซึ่งออกมาเป็น10ลำดับ ดังนี้

1. Interstellar
2. Minion
3. Guardians of the Galaxy
4. Deadpool
5. Mad Max:Fury Road
6. The Shawshank Redemption
7. The Dark Knight
8. Whiplash
9. The Godfather
10. Fight Club

และคำตอบที่ได้จากMachine Learning

1. Stiff Upper Lips
2. Me you and Five Bucks
3. Dancer , Texas Pop 81
4. Little Big Top
5. Sardaarji
6. One Man's Hero
7. There Goes My Baby
8. The Shawshank Redemption
9. The Prisoner of Zenda
10. The Godfather